In [27]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.spritmonitor.de/en/detail/679341.html'

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the details of the vehicle
details = soup.find('div', {'id': 'vehicledetails'})
vehicle = details.find('h1').text.strip().split(' - ')
manufacturer = vehicle[0]
model = vehicle[1]
version = vehicle[2]

table = soup.find('table', {'class': 'itemtable'})
rows = table.find_all('tr')

# Find the engine power of the vehicle
engine_power = None
engine_power_elem = details.find(text='Engine power:')
if engine_power_elem:
    engine_power = engine_power_elem.find_next('span').text.strip().split(' ')[0]

quantitys = [] 
Motor_ways = []
Citys = []
Country_roads = []
tire_types = []
trips = []
styles = []
consumptions = []
Consumptionsperkiloms = []
Average_speeds = []
quantitys_2 = []
Parking_heats = []
fuelprices = []

In [28]:
vehicle

['Volkswagen ', 'Golf', 'e-Golf']

In [29]:
row = rows[20]
row

<tr>
<td class="fueldate">20.05.2019</td>
<td class="fuelkmpos"></td>
<td class="trip" onmouseout="hideTooltip()" onmouseover="showTooltip('1,0 km')">    		  
    		        		    	1,0    		        		</td>
<td class="quantity" onmouseout="hideTooltip()" onmouseover="showTooltip('0,11 kWh')">
    		        		    	0,11    		        		</td>
<td class="fuelsort" onmouseout="hideTooltip()" onmouseover="showTooltip('Electricity')">
<img src="/sc/pics/v2086/fuelsort/fuelsort_19.png"/><br/>
</td>
<td class="tire">
<img alt="Winter tires" border="0" onmouseout="hideTooltip()" onmouseover="showTooltip('Winter tires')" src="/sc/pics/v2086/vdetail/tire_2.png"/> <br/>
</td>
<td class="street">
<img alt="City" onmouseout="hideTooltip()" onmouseover="showTooltip('City')" src="/sc/pics/v2086/vdetail/street_2.png"/> </td>
<td class="style">
<img alt="Moderate" border="0" onmouseout="hideTooltip()" onmouseover="showTooltip('Moderate')" src="/sc/pics/v2086/vdetail/style_1.png"/> </td>
<td class="fuelprice

In [30]:
page_number = 1
while page_number<91:
  this_page = (url + '?page=%d') % page_number
  response = requests.get(this_page)
  soup = BeautifulSoup(response.content, 'html.parser')
  table = soup.find('table', {'class': 'itemtable'})
  rows = table.find_all('tr')
  for row in rows:

    quantity = [td.text.strip() if td.text.strip() else None for td in row.find_all('td',{'class':'quantity'})]
    quantitys.append(quantity)

    tire_type = [td.find('img', alt=True)['alt'] if td.find('img', alt=True)['alt'] else None for td in row.find_all('td',{'class':'tire'})]
    tire_types.append(tire_type)

    trip = [td.text.strip() if td.text.strip() else None for td in row.find_all('td',{'class':'trip'})]
    trips.append(trip)

    style = [td.find('img', alt=True)['alt'] if td.find('img', alt=True)['alt'] else None for td in row.find_all('td',{'class':'style'})]
    styles.append(style)

    if row.find('td',{'class':'street'}) is not None:
      street = row.find('td',{'class':'street'}).find_all('img', alt=True)
      motor_way = 0
      country_road = 0
      city = 0
      for s in street:
        if s['alt'] == 'Motor-way':
          motor_way = 1
        if s['alt'] == 'Country roads':
          country_road = 1
        if s['alt'] == 'City':
          city = 1
      Motor_ways.append(motor_way)
      Country_roads.append(country_road)
      Citys.append(city)
    else:
      Motor_ways.append(None)
      Country_roads.append(None)
      Citys.append(None)

    if row.find('td',{'class':'consumption'}) is not None:
      consumption = row.find('td',{'class':'consumption'}).text.strip().replace(',','.')
      consumptions.append(consumption)
    else:
      consumptions.append(None)

    if row.find('td',{'class':'fuelprice'}) is not None:
      fuelprice = row.find('td',{'class':'fuelprice'}).text.strip().replace(',','.')
      fuelprices.append(fuelprice)
    else:
      fuelprices.append(None)
    fuelprices
    
    if row.find('td',{'class':'fuelnote'}) is not None:
      fuel_note = row.find('td',{'class':'fuelnote'}).find_all('img', alt=True)
      for f in fuel_note:
        if f['onmouseover'].find('Consumption: ') != -1:
          start_Consumption = f['onmouseover'].find('Consumption: ')
          end_Consumption = f['onmouseover'].find(' kWh/100km')
          consumptionper = f['onmouseover'][start_Consumption+len('Consumption: '):end_Consumption]
          consumptionper = float(consumptionper.replace(',', '.'))
        if f['onmouseover'].find('Avg speed: ') != -1:
          start_Consumption = f['onmouseover'].find('Avg speed: ')
          end_Consumption = f['onmouseover'].find(' km/h')
          average_speed = f['onmouseover'][start_Consumption+len('Avg speed: '):end_Consumption]
          average_speed = float(average_speed.replace(',', '.'))
        if f['onmouseover'].find('Quantity: ') != -1:
          start_Consumption = f['onmouseover'].find('Quantity: ')
          end_Consumption = f['onmouseover'].find(' Kilowatt hour')
          quantity_2 = f['onmouseover'][start_Consumption+len('Quantity: '):end_Consumption]
          quantity_2 = float(quantity_2.replace(',', '.'))
          
        if f == fuel_note[0] and f['alt'] != 'Park heating':
          Parking_heat = 0 
        elif f == fuel_note[0] and f['alt'] == 'Park heating':
          Parking_heat = 1
      Consumptionsperkiloms.append(consumptionper)
      Average_speeds.append(average_speed)
      Parking_heats.append(Parking_heat)
      quantitys_2.append(quantity_2)
    else:
      Consumptionsperkiloms.append(None)
      Average_speeds.append(None)
      Parking_heats.append(None)
      quantitys_2.append(None)
    
  page_number += 1

In [31]:
lists = [quantitys, Motor_ways, Citys, Country_roads, tire_types, trips, styles, consumptions,Average_speeds, quantitys_2, Parking_heats, fuelprices]

# find the length of each list and store it in a new list
lengths = [len(lst) for lst in lists]

print(lengths) # output: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

[2784, 2784, 2784, 2784, 2784, 2784, 2784, 2784, 2784, 2784, 2784, 2784]


In [32]:
import pandas as pd
dict = {'Energy_1':quantitys,'Energy_2':quantitys_2,'Trip':trips,'Average_Speeds':Average_speeds,'Style':styles,'Motor_Way':Motor_ways, 'City':Citys, 'Country_Road':Country_roads,
             'Tire_Type':tire_types,'Parking_Heat':Parking_heats, 'Consumption_1':consumptions,
             'Consumption_2':Consumptionsperkiloms, 'Price':fuelprices} 
    
df = pd.DataFrame(dict)

In [33]:
df

,Energy_1,Energy_2,Trip,Average_Speeds,Style,Motor_Way,City,Country_Road,Tire_Type,Parking_Heat,Consumption_1,Consumption_2,Price
0,[],NaN,[],NaN,[],NaN,NaN,NaN,[],NaN,None,NaN,None
1,"[2,81]",2.80,"[23,0]",31.0,[Moderate],0.0,1.0,1.0,[Summer tires],0.0,12.22,12.2,0.01
2,"[1,94]",1.90,"[18,0]",47.0,[Moderate],0.0,1.0,0.0,[Summer tires],0.0,10.78,10.8,0.01
3,"[0,44]",0.40,"[4,0]",21.0,[Moderate],0.0,1.0,0.0,[Summer tires],0.0,11.00,11.1,0.01
4,"[0,29]",0.30,"[2,0]",18.0,[Moderate],0.0,1.0,0.0,[Summer tires],0.0,14.50,14.3,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,"[5,62]",0.81,"[26,0]",45.0,[Normal],1.0,1.0,1.0,[Winter tires],0.0,21.62,21.6,
2780,"[14,25]",0.81,"[77,0]",42.0,[Normal],1.0,1.0,1.0,[Winter tires],0.0,18.51,18.5,
2781,"[4,97]",0.81,"[26,0]",23.0,[Normal],1.0,1.0,1.0,[Winter tires],0.0,19.12,19.1,
2782,"[4,76]",0.81,"[20,0]",46.0,[Fast],1.0,1.0,1.0,[Winter tires],0.0,23.80,23.8,0.01


In [34]:
df.dtypes

Energy_1           object
Energy_2          float64
Trip               object
Average_Speeds    float64
Style              object
Motor_Way         float64
City              float64
Country_Road      float64
Tire_Type          object
Parking_Heat      float64
Consumption_1      object
Consumption_2     float64
Price              object
dtype: object

In [36]:
from google.colab import files
df.to_csv('data_Volkswagen_with_fuelprice.csv', encoding = 'utf-8-sig') 
files.download('data_Volkswagen_with_fuelprice.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>